In [1]:
import pymysql


In [2]:
#performing a principal component analysis on earnings information

sql = '''SELECT act_symbol
FROM balance_sheet_assets
INTERSECT
SELECT act_symbol
FROM balance_sheet_liabilities
INTERSECT

INTERSECT
SELECT act_symbol
FROM income_statement;
'''

class DB:
    def __init__(self):
        self.user = "root"
        self.port = 8000
        self.db = "earnings"
        self.connection = pymysql.connect(host="127.0.0.1",
                                     port=self.port,
                                     user=self.user,
                                     db=self.db)

    def __call__(self, query):
        with self.connection.cursor() as cursor:
            cursor.execute(query)
            data = cursor.fetchall()
        return data
    
db = DB()

In [3]:
#Some descr
bsa = set(db("SELECT DISTINCT act_symbol FROM balance_sheet_assets"))
cf = set(db("SELECT DISTINCT act_symbol FROM cash_flow_statement"))
ist = set(db("SELECT DISTINCT act_symbol FROM income_statement"))
eps = set(db("SELECT DISTINCT act_symbol FROM eps_estimate"))

# Find the intersection of all three sets
described = bsa.intersection(cf, ist, eps)

# Output the result
print("Number of fully described stocks:", len(described))
#print("Fully described stocks:", described)

blacklist = bsa.union(cf, ist, eps) - described
print("Number of stocks with incomplete data:", len(blacklist))


Number of fully described stocks: 6183
Number of stocks with incomplete data: 2395


In [4]:
symbols = [s[0] for s in described]

In [5]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt

def pca3(df, df_labels):
    # Load the dataset and labels

    # Data Cleaning (if necessary)
    # df.fillna(method='ffill', inplace=True)  # Example: Forward fill to handle missing values

    # Data Cleaning (if necessary)
    # df.fillna(method='ffill', inplace=True)  # Example: Forward fill to handle missing values

    # Exploratory Data Analysis (EDA)
    #print(df.describe())

    # Correlation Analysis
    correlation_matrix = df.corr()
    #print("Correlation Matrix:\n", correlation_matrix)

    # Plot the heatmap for correlation matrix
    #plt.figure(figsize=(12, 8))
    #sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
    #plt.title('Correlation Matrix Heatmap')
    #plt.show()

    # Perform PCA for dimensionality reduction
    pca = PCA(n_components=len(df.columns))
    pca.fit(df)
    explained_variance = pca.explained_variance_ratio_

    # Plot explained variance
    #plt.figure(figsize=(8, 5))
    #plt.plot(range(1, len(df.columns) + 1), np.cumsum(explained_variance), marker='o', linestyle='--')
    #plt.title('Explained Variance by Principal Components')
    #plt.xlabel('Number of Components')
    #plt.ylabel('Cumulative Explained Variance')
    #plt.show()

    # Select components that explain up to 95% of variance
    cumulative_variance = np.cumsum(explained_variance)
    n_components = np.argmax(cumulative_variance >= 0.99) + 1
    #print(f"Number of components explaining 95% variance: {n_components}")

    # Get the loadings of the selected components
    loadings = pca.components_[:n_components]

    # Sum the absolute values of loadings across selected components for each feature
    importance = np.sum(np.abs(loadings), axis=0)

    # Get the indices of the most important features
    important_indices = np.argsort(importance)[::-1]

    # Print the most important features based on df_labels
    selected_features = [df_labels[i] for i in important_indices[:n_components]]
    #print("Selected Features by PCA:\n", selected_features)
    return selected_features

In [6]:
#!pip install scikit-learn matplotlib
#!pip install tqdm

In [7]:
#init pca stats. 
import tqdm
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

first = []
significance = []
freq = dict()
query = """
SELECT 
    bsa.date, 
    bsa.*, 
    bse.*,
    bsl.*
FROM 
    balance_sheet_assets bsa
JOIN 
    balance_sheet_equity bse ON bsa.act_symbol = bse.act_symbol AND bsa.date = bse.date
JOIN 
    balance_sheet_liabilities bsl ON bsa.act_symbol = bsl.act_symbol AND bsa.date = bsl.date
WHERE 
    bsa.date > '2019-12-31' AND
    bsa.period = 'Quarter' AND
    bse.period = 'Quarter' AND
    bsl.period = 'Quarter' 
ORDER BY 
    bsa.date
LIMIT 10000 OFFSET offset;
"""
'''
JOIN 
    cash_flow_statement cfs ON bsa.act_symbol = cfs.act_symbol AND bsa.date = cfs.date
JOIN 
    income_statement ist ON bsa.act_symbol = cfs.act_symbol AND bsa.date = ist.date
    cfs.*,
    ist.*
JOIN 
    eps_estimate eps ON bsa.act_symbol = eps.act_symbol AND bsa.date = eps.date
    eps.*, 
bsa.act_symbol = 'SYMBOL' AND
JOIN 
    eps_estimate eps ON bsa.act_symbol = eps.act_symbol AND bsa.date = eps.date


'''
#LIMIT 10000;
row = "| date       | date       | act_symbol | period  | cash_and_equivalents | receivables | notes_receivable | inventories | other_current_assets | total_current_assets | net_property_and_equipment | investments_and_advances | other_non_current_assets | deferred_charges | intangibles | deposits_and_other_assets | total_assets | act_symbol | date       | period  | preferred_stock | common_stock | capital_surplus | retained_earnings | other_equity | treasury_stock | total_equity | total_liabilities_and_equity | shares_outstanding | book_value_per_share | act_symbol | date       | period  | notes_payable | accounts_payable | current_portion_long_term_debt | current_portion_capital_leases | accrued_expenses | income_taxes_payable | other_current_liabilities | total_current_liabilities | mortgages | deferred_taxes_or_income | convertible_debt | long_term_debt | non_current_capital_leases | other_non_current_liabilities | minority_interest | total_liabilities |"


In [8]:
'''for o in tqdm.tqdm(range(0, 84000, 1000)):
    s = "l"
    data = db(query.replace("SYMBOL", s).replace("offset", str(o)))
    labels = row.replace(" ", "").split("|")[1:-1]
    refined_labels = [l for l in labels if l not in ["date", "act_symbol", "period"]]
    #print(labels)
    df_matrix = np.array(data).transpose().tolist()
    df_matrix_dict = {
        l:df_matrix[i] for i, l in enumerate(labels) if l not in ["date", "act_symbol", "period"]
    }
    df = pd.DataFrame(df_matrix_dict)
    selected_features = pca3(df, refined_labels)
    freq[str(o)] = selected_features
'''

'for o in tqdm.tqdm(range(0, 84000, 1000)):\n    s = "l"\n    data = db(query.replace("SYMBOL", s).replace("offset", str(o)))\n    labels = row.replace(" ", "").split("|")[1:-1]\n    refined_labels = [l for l in labels if l not in ["date", "act_symbol", "period"]]\n    #print(labels)\n    df_matrix = np.array(data).transpose().tolist()\n    df_matrix_dict = {\n        l:df_matrix[i] for i, l in enumerate(labels) if l not in ["date", "act_symbol", "period"]\n    }\n    df = pd.DataFrame(df_matrix_dict)\n    selected_features = pca3(df, refined_labels)\n    freq[str(o)] = selected_features\n'

In [9]:
from collections import defaultdict
    
def frequency_scores(freq):
    # Initialize a defaultdict to count occurrences
    frequency = defaultdict(int)

    # Loop through each key-value pair in freq
    for key, labels in freq.items():
        for index, label in enumerate(labels):
            # Calculate golf-style score (10 points for first, 9 for second, etc.)
            score = 10 - index
            frequency[label] += score

    # Convert defaultdict to regular dictionary for sorting
    frequency_dict = dict(frequency)

    # Sort the labels by their total score in descending order
    sorted_labels = sorted(frequency_dict, key=frequency_dict.get, reverse=True)

    # Print frequency histogram with golf-style scores
    print("Frequency Histogram with Golf-style Scores:")
    for label in sorted_labels:
        print(f"{label}: {frequency_dict[label]}")


In [12]:
print(" === Balance Sheet === ")
balancesheet_sql = '''
SELECT 
    bsa.date, bsa.*, bse.*, bsl.*
FROM 
    balance_sheet_assets bsa JOIN balance_sheet_equity bse ON bsa.act_symbol = bse.act_symbol AND bsa.date = bse.date JOIN balance_sheet_liabilities bsl ON bsa.act_symbol = bsl.act_symbol AND bsa.date = bsl.date
WHERE 
    bsa.date > '2019-12-31' AND bsa.period = 'Quarter' AND bse.period = 'Quarter' AND bsl.period = 'Quarter' 
ORDER BY 
    bsa.date
LIMIT 10000 OFFSET offset;'''
bs_row = '| date       | date       | act_symbol | period  | cash_and_equivalents | receivables | notes_receivable | inventories | other_current_assets | total_current_assets | net_property_and_equipment | investments_and_advances | other_non_current_assets | deferred_charges | intangibles | deposits_and_other_assets | total_assets | act_symbol | date       | period  | preferred_stock | common_stock | capital_surplus | retained_earnings | other_equity | treasury_stock | total_equity | total_liabilities_and_equity | shares_outstanding | book_value_per_share | act_symbol | date       | period  | notes_payable | accounts_payable | current_portion_long_term_debt | current_portion_capital_leases | accrued_expenses | income_taxes_payable | other_current_liabilities | total_current_liabilities | mortgages | deferred_taxes_or_income | convertible_debt | long_term_debt | non_current_capital_leases | other_non_current_liabilities | minority_interest | total_liabilities |'
bs_freq = {}
try:
    for o in tqdm.tqdm(range(0, 84000, 10000)):
        data = db(balancesheet_sql.replace("offset", str(o)))
        labels = bs_row.replace(" ", "").split("|")[1:-1]
        refined_labels = [l for l in labels if l not in ["date", "act_symbol", "period"]]
        df_matrix = np.array(data).transpose().tolist()
        df_matrix_dict = {
            l:df_matrix[i] for i, l in enumerate(labels) if l not in ["date", "act_symbol", "period"]
        }
        df = pd.DataFrame(df_matrix_dict)
        selected_features = pca3(df, refined_labels)
        bs_freq[str(o)] = selected_features
except:
    print(o)
finally:
    frequency_scores(bs_freq)
    
print(" === Cashflow Statement === ")
cashflow_sql = '''
SELECT 
    cfs.*
FROM 
    cash_flow_statement cfs
WHERE 
    cfs.date > '2019-12-31'
ORDER BY 
    cfs.date
LIMIT 10000 OFFSET offset;'''
cfs_row = '| act_symbol | date       | period | net_income | depreciation_amortization_and_depletion | net_change_from_assets | net_cash_from_discontinued_operations | other_operating_activities | net_cash_from_operating_activities | property_and_equipment | acquisition_of_subsidiaries | investments | other_investing_activities | net_cash_from_investing_activities | issuance_of_capital_stock | issuance_of_debt | increase_short_term_debt | payment_of_dividends_and_other_distributions | other_financing_activities | net_cash_from_financing_activities | effect_of_exchange_rate_changes | net_change_in_cash_and_equivalents | cash_at_beginning_of_period | cash_at_end_of_period | diluted_net_eps |'
cfs_freq = {}
try:
    for o in tqdm.tqdm(range(0, 42000, 10000)):
        data = db(cashflow_sql.replace("offset", str(o)))
        labels = cfs_row.replace(" ", "").split("|")[1:-1]
        refined_labels = [l for l in labels if l not in ["date", "act_symbol", "period"]]
        df_matrix = np.array(data).transpose().tolist()
        df_matrix_dict = {
            l:df_matrix[i] for i, l in enumerate(labels) if l not in ["date", "act_symbol", "period"]
        }
        df = pd.DataFrame(df_matrix_dict)
        selected_features = pca3(df, refined_labels)
        cfs_freq[str(o)] = selected_features
except:
    print(o)
finally:
    frequency_scores(cfs_freq)

print(" === Income Statement === ")
cashflow_sql = '''
SELECT 
    ics.*
FROM 
    income_statement ics
WHERE 
    ics.date > '2019-12-31'
ORDER BY 
    ics.date
LIMIT 10000 OFFSET offset;'''
ics_row = '| act_symbol | date       | period | sales      | cost_of_goods | gross_profit | selling_administrative_depreciation_amortization_expenses | income_after_depreciation_and_amortization | non_operating_income | interest_expense | pretax_income | income_taxes | minority_interest | investment_gains | other_income | income_from_continuing_operations | extras_and_discontinued_operations | net_income | income_before_depreciation_and_amortization | depreciation_and_amortization | average_shares | diluted_eps_before_non_recurring_items | diluted_net_eps |'
ics_freq = {}
try:
    for o in tqdm.tqdm(range(0, 96000, 10000)):
        data = db(cashflow_sql.replace("offset", str(o)))
        labels = ics_row.replace(" ", "").split("|")[1:-1]
        refined_labels = [l for l in labels if l not in ["date", "act_symbol", "period"]]
        df_matrix = np.array(data).transpose().tolist()
        df_matrix_dict = {
            l:df_matrix[i] for i, l in enumerate(labels) if l not in ["date", "act_symbol", "period"]
        }
        df = pd.DataFrame(df_matrix_dict)
        df = df.fillna(0)
        selected_features = pca3(df, refined_labels)
        ics_freq[str(o)] = selected_features
except Exception as E:
    print(o)
finally:
    frequency_scores(ics_freq)
        

 === Balance Sheet === 


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:30<00:00,  3.40s/it]


Frequency Histogram with Golf-style Scores:
retained_earnings: 84
investments_and_advances: 75
total_current_assets: 61
accounts_payable: 48
cash_and_equivalents: 43
other_non_current_liabilities: 40
receivables: 39
net_property_and_equipment: 34
intangibles: 24
long_term_debt: 17
total_current_liabilities: 12
total_liabilities: 6
other_non_current_assets: 6
total_equity: 3
other_current_liabilities: 0
 === Cashflow Statement === 


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.20s/it]


Frequency Histogram with Golf-style Scores:
net_change_from_assets: 33
other_financing_activities: 31
net_cash_from_financing_activities: 28
net_income: 24
investments: 23
other_investing_activities: 19
other_operating_activities: 18
cash_at_beginning_of_period: 17
net_change_in_cash_and_equivalents: 14
net_cash_from_investing_activities: 13
net_cash_from_operating_activities: 12
increase_short_term_debt: 10
cash_at_end_of_period: 10
 === Income Statement === 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:14<00:00,  1.44s/it]

Frequency Histogram with Golf-style Scores:
cost_of_goods: 94
selling_administrative_depreciation_amortization_expenses: 80
gross_profit: 69
income_before_depreciation_and_amortization: 54
pretax_income: 22


In [ ]:
#Feature Selection
# === Balance Sheet === 

#accounts_payable
#total_current_assets
#retained_earnings
#net property
#investments_and_advances
#net_property_and_equipment
#total_liabilities

# === Cashflow Statement === 

#cash_at_beginning_of_period: 190
#net_cash_from_investing_activities: 186
#net_cash_from_financing_activities: 182
#net_cash_from_operating_activities: 163
#net_change_from_assets: 162
#other_financing_activities: 155
#investments: 146
#cash_at_end_of_period: 136
#net_income: 132
#other_operating_activities: 104

# === Income Statement === 

#cost_of_goods
#selling_administrative_depreciation_amortization_expenses
#gross_profit


In [11]:
''' 
=== Balance Sheet === 
Frequency Histogram with Golf-style Scores:
accounts_payable: 569
total_current_assets: 524
retained_earnings: 445
investments_and_advances: 432
net_property_and_equipment: 375
receivables: 309
intangibles: 307
other_non_current_liabilities: 279
total_current_liabilities: 268
cash_and_equivalents: 138
total_liabilities: 129
other_current_liabilities: 119
total_equity: 108
other_non_current_assets: 75
deposits_and_other_assets: 54
capital_surplus: 49
long_term_debt: 42
treasury_stock: 10
other_current_assets: 8

 === Cashflow Statement === 
Frequency Histogram with Golf-style Scores:
cash_at_beginning_of_period: 190
net_cash_from_investing_activities: 186
net_cash_from_financing_activities: 182
net_cash_from_operating_activities: 163
net_change_from_assets: 162
other_financing_activities: 155
investments: 146
cash_at_end_of_period: 136
net_income: 132
other_operating_activities: 104
net_change_in_cash_and_equivalents: 80
issuance_of_debt: 75
other_investing_activities: 58
increase_short_term_debt: 44
depreciation_amortization_and_depletion: 35
acquisition_of_subsidiaries: 28
issuance_of_capital_stock: 18
property_and_equipment: 17
net_cash_from_discontinued_operations: 7
payment_of_dividends_and_other_distributions: 2

 === Income Statement === 
Frequency Histogram with Golf-style Scores:
cost_of_goods: 921
selling_administrative_depreciation_amortization_expenses: 611
gross_profit: 596
sales: 281
income_before_depreciation_and_amortization: 160
average_shares: 151
income_after_depreciation_and_amortization: 51
pretax_income: 43
net_income: 17
'''

In [ ]:
import yfinance
for s in 